In [13]:
import json

with open(r"D:\BEHAVIOR-1K\asset_pipeline\artifacts\pipeline\object_inventory.json", "r") as f:
    provider_map = json.load(f)["providers"]
    providers = {x.split("-")[1]: y for x, y in provider_map.items()}
    categories = {x.split("-")[1]: x.split("-")[0] for x, y in provider_map.items()}

In [14]:
import csv

# Load the rename file
RENAMES = {}
with open("D:/BEHAVIOR-1K/asset_pipeline/metadata/object_renames.csv") as f:
    for row in csv.DictReader(f):
        key = (row["ID (auto)"])
        RENAMES[key] = row["New Category"]

# Load the deletion file
DELETION_QUEUE = set()
with open("D:/BEHAVIOR-1K/asset_pipeline/metadata/deletion_queue.csv", "r") as f:
    for row in csv.DictReader(f):
        DELETION_QUEUE.add(row["Object"].strip().split("-")[1])


def maybe_rename_category(cat, model):
    if model in RENAMES:
        return RENAMES[model]
    return cat

for x, cat in list(categories.items()):
    categories[x] = maybe_rename_category(cat, x)

for x in DELETION_QUEUE:
    if x in categories:
        del categories[x]

In [15]:
# For each file in the TODOs, get the list of relevant objects 
def get_objects_from_todo_file(file_stem):
    with open(f"D:/BEHAVIOR-1K/asset_pipeline/{file_stem}.txt", "r") as f:
        objects = {x.strip() for x in f.readlines()}
        assert all(len(o) == 6 for o in objects), f"Some object IDs in {file_stem} are not 6 characters long"
        return set(objects)
    
links_to_complaints = [
    (get_objects_from_todo_file("todo-appearance"), "STRUCTURE-APPEARANCE: Something is wrong with how this object looks in simulation."),
    (get_objects_from_todo_file("todo-category"), "STRUCTURE-SYNSET: This object is assigned to a wall/floor category but it may need to either be attached to a structure object or moved into another category."),
    (get_objects_from_todo_file("todo-synset"), "STRUCTURE-SYNSET: This object is assigned to a wall/floor category but it may need to either be attached to a structure object or moved into another category."),
    (get_objects_from_todo_file("todo-glassness"), "STRUCTURE-GLASSNESS: This object looks like it might be glass in 3ds Max but is not annotated as glass."),
    (get_objects_from_todo_file("todo-multiple-pieces"), "STRUCTURE-MULTIPLE-PIECES: This object is made up of multiple pieces - it should be broken apart. Please consult w/ Cem on how to do this for things like pavers - we want those to be together, but we want large, disjoint flat pieces of floor to be separate objects."),
    (get_objects_from_todo_file("todo-thickness"), "STRUCTURE-THICKNESS: This object is a paper-thin floor or wall that should have some thickness."),
    (get_objects_from_todo_file("todo-triangulation"), "STRUCTURE-TRIANGULATION: This object seems to suffer from triangulation issues. Verify that this is fixed with new triangulator.", True),
    (get_objects_from_todo_file("todo-unclosed"), "STRUCTURE-UNCLOSED: This object is not a closed mesh: at least one of its sides is open when it should be a solid volume."),
]

In [16]:
# Add tippy lamps
def get_objects_by_category(cat):
  return [model for model, category in categories.items() if category == cat]

links_to_complaints.append(
  (get_objects_by_category("table_lamp") + get_objects_by_category("floor_lamp"), "CENTEROFMASS: This object is a lamp object. Those objects might have a tendency to tip over because the automatically generated center of mass does not represent the heavy base. Consider annotating a center of mass.")
)

In [17]:
# Add fillable object issues
fillable_objects = [
    ("bfaqfe", "fuel can cap is stuck"),
    ("gcixra", "oil bottle with no top opening"),
    ("ueagnt", "no proepr opening"),
    ("vlplhs", "specimen bottle cap stuck"),
    ("wklill", "bad joint"),
    ("yjmnej", "add sides so that fluids stay in"),
    ("ceaeqf", "put the bowl back together, add prismatic joint"),
    ("qornxa", "add prismatic joint, figure out how to make moving fillable"),
    ("svkdji", "hole keeps getting closed. add manual cmesh"),
    ("ignowl", "something wrong with the link here. does it have any meshes at all?"),
    ("adxzhe", "create two fillable volumes, use as test case"),
    ("nedrsh", "objects penetrate through the cmesh"),
]
for fillable_object in fillable_objects:
    links_to_complaints.append(
        ([fillable_object[0]], f"FILLABLE: {fillable_object[1]}")
    )

In [18]:
# Add other visual complaints
links_to_complaints.extend([
  ([x for x in providers.keys() if "todo" in x], "TODO: Object has the word TODO in its name, meaning it might have something wrong with it. Either fix it or replace it with another object."),
  (["acooui"], "APPEARANCE: This object has multiple almonds. Keep just one.")
])

In [19]:
# Assert that everything is in the providers list
for x in links_to_complaints:
    missing_providers = set(x[0]) - set(providers.keys())
    assert not missing_providers, f"Missing providers: {missing_providers}"

In [20]:
# Toss all the complaints in the files
def add_complaint(model, complaint_msg, processed=False):
    model_target = providers[model]

    # Open the complaints file for the target
    complaints_file = f"D:/BEHAVIOR-1K/asset_pipeline/cad/" + model_target + "/complaints.json"
    with open(complaints_file, "r") as f:
        complaints = json.load(f)

    complaint = {
        "object": categories[model] + "-" + model,
        "message": complaint_msg,
        "complaint": "See message",
        "processed": processed,
        "new": True,
    }

    # Add the complaint
    complaints.append(complaint)

    # Write the complaints back
    with open(complaints_file, "w") as f:
        json.dump(complaints, f, indent=4)

In [21]:
from tqdm.notebook import tqdm

# Unroll the complaints
individual_complaints = []
for data in links_to_complaints:
    for model in data[0]:
        individual_complaints.append((model, *data[1:]))

for data in tqdm(individual_complaints):
    add_complaint(*data)

  0%|          | 0/467 [00:00<?, ?it/s]